In [1]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func
import datetime
import json
import pandas as pd
import numpy as np

In [2]:
#set up engine
engine = create_engine("sqlite:///../Data/Poker_data.db")

In [3]:
#Verify that the database is connected and get table names for reference
inspector = inspect(engine)
inspector.get_table_names()

['dbo_hand_roster', 'dbo_hands', 'dbo_players']

In [4]:
#Verify column names for queries and future use
columns = inspector.get_columns('dbo_players')
column_names = []
for c in columns:
    column_names.append(c['name'])
    print(c['name'], c["type"])

player_name TEXT
hand_id BIGINT
num_players INTEGER
chair_num INTEGER
action_pre_flop TEXT
action_flop TEXT
action_turn TEXT
action_river TEXT
chip_count INTEGER
player_bet INTEGER
player_winnings INTEGER
revealed_card_1 TEXT
revealed_card_2 TEXT


In [5]:
#Parameters to run test

In [6]:
#create placeholders for data

In [7]:
#run queries for data
hands_df = pd.read_sql('SELECT hand.num_players, hand.flop_pot, hand.turn_pot, hand.river_pot, hand.winnings_pot, player.chip_count, player.chair_num, player.player_bet, player.player_winnings FROM dbo_hands hand JOIN dbo_players player ON hand.hand_id = player.hand_id ORDER BY hand.hand_id DESC', engine)
hands_df.head(100)

,num_players,flop_pot,turn_pot,river_pot,winnings_pot,chip_count,chair_num,player_bet,player_winnings
0,2,0/0,0/0,0/0,0/20,1995,1,10,20
1,2,0/0,0/0,0/0,0/20,50,2,10,0
2,2,2/20,2/20,0/0,1/1920,1045,2,970,1920
3,2,2/20,2/20,0/0,1/1920,1000,1,950,0
4,3,0/0,0/0,0/0,1/25,3550,1,5,0
...,...,...,...,...,...,...,...,...,...
95,4,2/25,0/0,0/0,1/35,782,2,10,0
96,4,3/60,2/80,2/120,2/160,2224,4,0,0
97,4,3/60,2/80,2/120,2/160,690,2,70,0
98,4,3/60,2/80,2/120,2/160,1165,1,20,0


In [8]:
strp_flop_df = hands_df
strp_flop_df['flop_pot'] = strp_flop_df['flop_pot'].str.split('/').str[1]
strp_flop_df['turn_pot'] = strp_flop_df['turn_pot'].str.split('/').str[1]
strp_flop_df['river_pot'] = strp_flop_df['river_pot'].str.split('/').str[1]
strp_flop_df['winnings_pot'] = strp_flop_df['winnings_pot'].str.split('/').str[1]
strp_flop_df

,num_players,flop_pot,turn_pot,river_pot,winnings_pot,chip_count,chair_num,player_bet,player_winnings
0,2,0,0,0,20,1995,1,10,20
1,2,0,0,0,20,50,2,10,0
2,2,20,20,0,1920,1045,2,970,1920
3,2,20,20,0,1920,1000,1,950,0
4,3,0,0,0,25,3550,1,5,0
...,...,...,...,...,...,...,...,...,...
139713,9,20,40,80,120,1070,2,0,0
139714,9,20,40,80,120,885,7,0,0
139715,9,20,40,80,120,1740,1,60,0
139716,9,20,40,80,120,855,6,0,0


In [9]:
#machine learning to estimate player winnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import pandas as pd
import tensorflow as tf

In [10]:
X = strp_flop_df.drop(['player_winnings'], axis=1)
y = strp_flop_df['player_winnings']
new_y = []
for y_t in y:
    if y_t > 0:
        y_t = 1
    else:
        y_t = 0
    new_y.append(y_t)
y = np.array(new_y)

In [11]:
X.head()

,num_players,flop_pot,turn_pot,river_pot,winnings_pot,chip_count,chair_num,player_bet
0,2,0,0,0,20,1995,1,10
1,2,0,0,0,20,50,2,10
2,2,20,20,0,1920,1045,2,970
3,2,20,20,0,1920,1000,1,950
4,3,0,0,0,25,3550,1,5


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [13]:
scaler = StandardScaler().fit(X_train)

In [14]:
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [15]:
# Define the deep learning model 
nn_model = tf.keras.models.Sequential()
nn_model.add(tf.keras.layers.Dense(units=8, activation="relu", input_dim=8))
nn_model.add(tf.keras.layers.Dense(units=8, activation="relu"))
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/50
3275/3275 [==============================] - 3s 735us/step - loss: 0.2987 - accuracy: 0.8536
Epoch 2/50
3275/3275 [==============================] - 2s 756us/step - loss: 0.1976 - accuracy: 0.9050
Epoch 3/50
3275/3275 [==============================] - 2s 760us/step - loss: 0.1777 - accuracy: 0.9074
Epoch 4/50
3275/3275 [==============================] - 2s 751us/step - loss: 0.1712 - accuracy: 0.9080
Epoch 5/50
3275/3275 [==============================] - 2s 760us/step - loss: 0.1684 - accuracy: 0.9081
Epoch 6/50
3275/3275 [==============================] - 2s 762us/step - loss: 0.1662 - accuracy: 0.9080
Epoch 7/50
3275/3275 [==============================] - 2s 761us/step - loss: 0.1643 - accuracy: 0.9080
Epoch 8/50
3275/3275 [==============================] - 2s 759us/step - loss: 0.1631 - accuracy: 0.9082
Epoch 9/50
3275/3275 [==============================] - 3s 764us/step - loss: 0.1622 - accuracy: 0.9077
Epoch 10/50
3275/3275 [==============================] - 2s 753u

In [16]:
type(nn_model)

keras.engine.sequential.Sequential

In [17]:
len(y_train)

104788

In [23]:
x_predict = [9,20,40,80,120,1070,2,0]
x_predict = np.array([x_predict])
x_predict.flatten()
x_predict.shape

(1, 8)

In [24]:
y_predict = nn_model.predict(x_predict)

1/1 [==============================] - 0s 12ms/step


In [25]:
nn_model.evaluate(x_predict,y_predict,verbose=1)

1/1 [==============================] - 0s 13ms/step - loss: 0.0000e+00 - accuracy: 1.0000


[0.0, 1.0]

In [26]:
y_predict

array([[0.]], dtype=float32)

In [28]:
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 72        
                                                                 
 dense_1 (Dense)             (None, 8)                 72        
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 153
Trainable params: 153
Non-trainable params: 0
_________________________________________________________________


In [29]:
nn_model.save('poker_model.hdf5')